In [145]:
import numpy as np
from matplotlib import pyplot as plt

import os
import glob
import cv2

In [146]:
PIXELSIZE = 32 #28, 32
PIXELTIMES = 1024 # 784, 1024

In [147]:
root_dir = "asl_alphabet_train"
image_data = {}

In [148]:
for class_folder in os.listdir(root_dir):
    class_path = os.path.join(root_dir, class_folder)
    if os.path.isdir(class_path):
        class_images = []
        for image_file in glob.glob(os.path.join(class_path, "*.jpg")):  # Change the file extension as needed
            class_images.append(image_file)
        image_data[class_folder] = class_images

In [149]:
for class_name, images in image_data.items():
    print(f"Class: {class_name}\tCount: {len(images)}")

Class: A	Count: 3000
Class: B	Count: 3000
Class: C	Count: 3000
Class: D	Count: 3000
Class: del	Count: 3000
Class: E	Count: 3000
Class: F	Count: 3000
Class: G	Count: 3000
Class: H	Count: 3000
Class: I	Count: 3000
Class: J	Count: 3000
Class: K	Count: 3000
Class: L	Count: 3000
Class: M	Count: 3000
Class: N	Count: 3000
Class: nothing	Count: 3000
Class: O	Count: 3000
Class: P	Count: 3000
Class: Q	Count: 3000
Class: R	Count: 3000
Class: S	Count: 3000
Class: space	Count: 3000
Class: T	Count: 3000
Class: U	Count: 3000
Class: V	Count: 3000
Class: W	Count: 3000
Class: X	Count: 3000
Class: Y	Count: 3000
Class: Z	Count: 3000


In [150]:
max_images_per_class = 110  # Set the maximum number of images per class
classNumber = 0

images = []
classes = []

for class_name, image_paths in image_data.items():
    class_count = 0  # Initialize a counter for the current class
    for image_path in image_paths:
        if isinstance(image_path, str) and os.path.exists(image_path):
            image = cv2.imread(image_path, 0)
            image = cv2.resize(image, (PIXELSIZE, PIXELSIZE), interpolation= cv2.INTER_LINEAR)

            if image is not None:  # Check if the image was successfully loaded
                images.append(image)
                classes.append(classNumber)
                class_count += 1
                
            if class_count >= max_images_per_class:
                break  # Stop loading images for the current class

    classNumber += 1

In [151]:
import random

# Combine the two lists into pairs
combined_lists = list(zip(classes, images))

# Shuffle the pairs
random.shuffle(combined_lists)

# Unzip the shuffled pairs back into separate lists
classes, images = zip(*combined_lists)

In [152]:
classes = np.array(classes)
images = np.array(images)

In [153]:
images = images.reshape(-1, PIXELTIMES)

In [154]:
m, n = images.shape
print(m, n)

3190 1024


In [155]:
Y_data= classes.T
X_data = images.T

X_data = X_data / 255.

In [156]:
X_train = X_data[0:m]
y_train = Y_data[0:m]

X_test = X_data[3000:m]
y_test = Y_data[3000:m]

In [157]:
_,m_train = X_train.shape

In [158]:
print(X_train)
print(y_train)

[[0.49803922 0.38431373 0.35294118 ... 0.34509804 0.34901961 0.34117647]
 [0.50980392 0.39607843 0.35686275 ... 0.34117647 0.36862745 0.36470588]
 [0.51372549 0.38431373 0.33333333 ... 0.35686275 0.36862745 0.37647059]
 ...
 [0.65098039 0.49803922 0.47843137 ... 0.45882353 0.45882353 0.49019608]
 [0.67843137 0.5372549  0.49019608 ... 0.48235294 0.44705882 0.48627451]
 [0.70588235 0.54509804 0.49411765 ... 0.47843137 0.47843137 0.51764706]]
[ 5 16 28 ... 25 26 24]


In [159]:
def init_params():
    W1 = np.random.rand(PIXELSIZE, PIXELTIMES) - 0.5
    b1 = np.random.rand(PIXELSIZE, 1) - 0.5
    W2 = np.random.rand(PIXELSIZE, PIXELSIZE) - 0.5
    b2 = np.random.rand(PIXELSIZE, 1) - 0.5
    return W1, b1, W2, b2

def ReLU(Z):
    return np.maximum(Z, 0)

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A
    
def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

def ReLU_deriv(Z):
    return Z > 0

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, PIXELSIZE))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2    
    return W1, b1, W2, b2

In [162]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()

    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        
        if i % 100 == 0:
            predictions = get_predictions(A2)
            print("Iteration: ", i)
            print("Accuracy: " + str(get_accuracy(predictions, Y)))
            print("================================================")
    return W1, b1, W2, b2

In [163]:
W1, b1, W2, b2 = gradient_descent(X_train, y_train, 0.10, 10000)

Iteration:  0
[13 13 13 ... 13 13 13] [ 5 16 28 ... 25 26 24]
Accuracy: 0.03197492163009404
Iteration:  100
[20 15 15 ... 15 15 20] [ 5 16 28 ... 25 26 24]
Accuracy: 0.06739811912225706
Iteration:  200
[ 1 15 15 ... 15 15 14] [ 5 16 28 ... 25 26 24]
Accuracy: 0.11379310344827587
Iteration:  300
[ 5  4 23 ... 25 25 25] [ 5 16 28 ... 25 26 24]
Accuracy: 0.2225705329153605
Iteration:  400
[ 5 28 13 ... 20 20 20] [ 5 16 28 ... 25 26 24]
Accuracy: 0.2858934169278997
Iteration:  500
[ 5 18 13 ... 25 25 23] [ 5 16 28 ... 25 26 24]
Accuracy: 0.3871473354231975
Iteration:  600
[ 5 16 14 ... 27 20 20] [ 5 16 28 ... 25 26 24]
Accuracy: 0.33699059561128525
Iteration:  700
[ 5 21 14 ... 27 27 19] [ 5 16 28 ... 25 26 24]
Accuracy: 0.44388714733542317
Iteration:  800
[ 5 12 14 ... 27 18 24] [ 5 16 28 ... 25 26 24]
Accuracy: 0.39717868338557993
Iteration:  900
[ 5 21 14 ... 13 27 27] [ 5 16 28 ... 25 26 24]
Accuracy: 0.42382445141065833
Iteration:  1000
[ 5 21 14 ... 27 27 28] [ 5 16 28 ... 25 26 24]


In [ ]:
def make_predictions(X, W1, b1, W2, b2):
    _, _, _, A2 = forward_prop(W1, b1, W2, b2, X)
    predictions = get_predictions(A2)
    return predictions

def test_prediction(index, W1, b1, W2, b2):
    current_image = X_train[:, index, None]
    prediction = make_predictions(X_train[:, index, None], W1, b1, W2, b2)
    label = y_train[index]
    print("Prediction: ", prediction)
    print("Label: ", label)
    
    current_image = current_image.reshape((PIXELSIZE, PIXELSIZE)) * 255
    plt.gray()
    plt.imshow(current_image, interpolation='nearest')
    plt.show()

In [ ]:
test_prediction(0, W1, b1, W2, b2)
test_prediction(1, W1, b1, W2, b2)
test_prediction(2, W1, b1, W2, b2)
test_prediction(3, W1, b1, W2, b2)

In [ ]:
dev_predictions = make_predictions(X_train, W1, b1, W2, b2)
get_accuracy(dev_predictions, y_train)